In [57]:
import pandas as pd
import requests
import numpy as np

In [119]:
def getDataOfCompany(x,api_key):
        merged_df = pd.read_csv(f'{x}fundamental_data.csv')
        merged_df = merged_df[:57]
        merged_df['fiscalDateEnding'] = pd.to_datetime(merged_df['fiscalDateEnding'])
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={x}&outputsize=full&apikey={api_key}'
        r = requests.get(url)
        data = r.json()
        # print(data)
        list_current = []
        list_next = []
        for i,rows in merged_df.iterrows():
            if(i-1<0):
                list_next.append(0)
                time = rows['fiscalDateEnding'].strftime('%Y-%m-%d')
                # list_current.append(float(data['Time Series (Daily)'][time]['4. close']))
                if time in data['Time Series (Daily)']:
                    list_current.append(float(data['Time Series (Daily)'][time]['4. close']))
                else:
                    time = (rows['fiscalDateEnding'] - pd.DateOffset(days=4)).strftime('%Y-%m-%d')
                    if time in data['Time Series (Daily)']:
                        list_current.append(float(data['Time Series (Daily)'][time]['4. close']))
                    else:
                        time = (rows['fiscalDateEnding'] + pd.DateOffset(days=4)).strftime('%Y-%m-%d')
                        list_current.append(float(data['Time Series (Daily)'][time]['4. close']))               
            else:
                current_row = merged_df.iloc[i]
                next_row = merged_df.iloc[i-1]
                ##########For current row ###############
                time = rows['fiscalDateEnding'].strftime('%Y-%m-%d')
                if time in data['Time Series (Daily)']:
                    list_current.append(float(data['Time Series (Daily)'][time]['4. close']))
                else:
                    time = (rows['fiscalDateEnding'] - pd.DateOffset(days=4)).strftime('%Y-%m-%d')
                    if time in data['Time Series (Daily)']:
                        list_current.append(float(data['Time Series (Daily)'][time]['4. close']))
                    else:
                        time = (rows['fiscalDateEnding'] + pd.DateOffset(days=4)).strftime('%Y-%m-%d')
                        list_current.append(float(data['Time Series (Daily)'][time]['4. close']))
                ###################End Here###########################
                ###################For Next Row#######################
                time = (next_row['fiscalDateEnding']).strftime('%Y-%m-%d')
                if time in data['Time Series (Daily)']:
                    list_next.append(float(data['Time Series (Daily)'][time]['4. close']))
                else:
                    time = (next_row['fiscalDateEnding'] - pd.DateOffset(days=4)).strftime('%Y-%m-%d')
                    if time in data['Time Series (Daily)']:
                        list_next.append(float(data['Time Series (Daily)'][time]['4. close']))
                    else:
                        time = (next_row['fiscalDateEnding'] + pd.DateOffset(days=4)).strftime('%Y-%m-%d')
                        list_next.append(float(data['Time Series (Daily)'][time]['4. close']))
                ##################End Here##############################
        data_list_curr = pd.DataFrame(list_current, columns = ['Current Price'])
        data_list_next = pd.DataFrame(list_next, columns = ['Next Price'])
        merged_df['Current Price'] = data_list_curr
        merged_df['Next Price'] = data_list_next
        merged_df = merged_df.loc[1:,:]
        merged_df['Difference'] = merged_df['Next Price'] - merged_df['Current Price']
        merged_df['Action'] = merged_df['Difference'].apply(lambda x: 1 if x > 0 else 0)
        return merged_df

In [120]:
def calculateFundamentalRatios(merged_df):
    merged_df['Current Ratio'] = merged_df['totalCurrentAssets'].astype(float)/merged_df['totalCurrentLiabilities'].astype(float)
    merged_df['Quick Ratio'] = (merged_df['totalCurrentAssets'].astype(float) - merged_df['inventory'].astype(float) - merged_df['otherCurrentAssets'].astype(float))/merged_df['totalCurrentLiabilities'].astype(float)
    merged_df['Debt to Equity'] = merged_df['totalLiabilities'].astype(float)/merged_df['totalShareholderEquity'].astype(float)
    merged_df['Debt to Asset'] = merged_df['totalLiabilities'].astype(float)/merged_df['totalAssets'].astype(float)
    merged_df['Gross Margin'] = merged_df['grossProfit'].astype(float)/merged_df['totalRevenue'].astype(float)
    merged_df['Profit Margin'] = merged_df['incomeBeforeTax'].astype(float)/merged_df['totalRevenue'].astype(float)
    merged_df['Operating Margin'] = merged_df['ebit'].astype(float)/merged_df['totalRevenue'].astype(float)
    merged_df['Interest Coverage'] = (merged_df['interestExpense'].astype(float) + merged_df['incomeTaxExpense'].astype(float) + merged_df['netIncome_x'].astype(float))/merged_df['interestExpense'].astype(float)
    merged_df['Inventory Ratio'] = merged_df['costofGoodsAndServicesSold'].astype(float)/merged_df['inventory'].astype(float)
    merged_df['Return On StockHolder'] = merged_df['netIncome_x'].astype(float)/merged_df['totalShareholderEquity'].astype(float)
    merged_df['Free Cash Flow Conversion'] = (merged_df['operatingCashflow'].astype(float) - merged_df['capitalExpenditures'].astype(float))/merged_df['ebitda'].astype(float)
    return merged_df

In [115]:
def dropUselessValues(merged_df, company_name):
    final_data = merged_df.drop(columns = ['reportedCurrency', 'grossProfit', 'totalRevenue',
       'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome',
       'sellingGeneralAndAdministrative', 'researchAndDevelopment',
       'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome',
       'interestIncome', 'interestExpense', 'nonInterestIncome',
       'otherNonOperatingIncome', 'depreciation',
       'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense',
       'interestAndDebtExpense', 'netIncomeFromContinuingOperations',
       'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda','reportedCurrency', 'totalAssets',
       'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue',
       'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables',
       'totalNonCurrentAssets', 'propertyPlantEquipment',
       'accumulatedDepreciationAmortizationPPE', 'intangibleAssets',
       'intangibleAssetsExcludingGoodwill', 'goodwill', 'investments',
       'longTermInvestments', 'shortTermInvestments', 'otherCurrentAssets',
       'otherNonCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities',
       'currentAccountsPayable', 'deferredRevenue', 'currentDebt',
       'shortTermDebt', 'totalNonCurrentLiabilities',
       'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt',
       'longTermDebtNoncurrent', 'shortLongTermDebtTotal',
       'otherCurrentLiabilities', 'otherNonCurrentLiabilities',
       'totalShareholderEquity', 'treasuryStock', 'retainedEarnings',
       'commonStock', 'commonStockSharesOutstanding','reportedCurrency', 'operatingCashflow',
       'paymentsForOperatingActivities', 'proceedsFromOperatingActivities',
       'changeInOperatingLiabilities', 'changeInOperatingAssets',
       'depreciationDepletionAndAmortization', 'capitalExpenditures',
       'changeInReceivables', 'changeInInventory', 'profitLoss',
       'cashflowFromInvestment', 'cashflowFromFinancing',
       'proceedsFromRepaymentsOfShortTermDebt',
       'paymentsForRepurchaseOfCommonStock', 'paymentsForRepurchaseOfEquity',
       'paymentsForRepurchaseOfPreferredStock', 'dividendPayout',
       'dividendPayoutCommonStock', 'dividendPayoutPreferredStock',
       'proceedsFromIssuanceOfCommonStock',
       'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet',
       'proceedsFromIssuanceOfPreferredStock',
       'proceedsFromRepurchaseOfEquity', 'proceedsFromSaleOfTreasuryStock',
       'changeInCashAndCashEquivalents', 'changeInExchangeRate'])
    final_data = final_data.drop(columns = ['reportedCurrency_x','netIncome_x','reportedCurrency_y','netIncome_y',
                                        'reportedDate','estimatedEPS','surprise','surprisePercentage','reportTime'])
    final_data.drop(columns = ['fiscalDateEnding',
                               'Current Price','Next Price','Difference'], inplace = True)
    # final_data.dropna(inplace = True)
    final_data.to_csv(f'{company_name}fundamental_data_final.csv', index=False)
    return final_data

In [127]:
merged_df = getDataOfCompany('SAP','W2ROFUHOJ9VQTHJ2')
merged_df

,fiscalDateEnding,reportedCurrency_x,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,Current Price,Next Price,Difference,Action
1,2023-12-31,EUR,6.208000e+09,8.468000e+09,2.260000e+09,2.260000e+09,1.985000e+09,353000000.0,1.667000e+09,4.223000e+09,...,2024-01-23,1.41,1.58,-0.17,-10.7595,post-market,155.90,197.09,41.19,1
2,2023-09-30,EUR,5.636000e+09,7.744000e+09,2.108000e+09,2.108000e+09,1.724000e+09,327000000.0,1.515000e+09,3.949000e+09,...,2023-10-18,1.45,1.36,0.09,6.6176,post-market,127.18,155.90,28.72,1
3,2023-06-30,EUR,5.410000e+09,7.555000e+09,2.145000e+09,2.145000e+09,1.358000e+09,322000000.0,1.565000e+09,4.055000e+09,...,2023-07-20,3.14,1.16,1.98,170.6897,post-market,136.81,127.18,-9.63,0
4,2023-03-31,EUR,5.285000e+09,7.441000e+09,2.156000e+09,2.156000e+09,1.230000e+09,348000000.0,1.573000e+09,4.220000e+09,...,2023-04-21,1.27,1.10,0.17,15.4545,pre-market,126.55,136.81,10.26,1
5,2022-12-31,EUR,6.064000e+09,8.436000e+09,2.372000e+09,2.372000e+09,1.328000e+09,517000000.0,1.621000e+09,4.330000e+09,...,2023-01-26,0.99,1.37,-0.38,-27.7372,pre-market,103.34,126.55,23.21,1
6,2022-09-30,EUR,5.575000e+09,7.841000e+09,2.266000e+09,2.266000e+09,1.239000e+09,521000000.0,1.574000e+09,4.344000e+09,...,2022-10-17,1.11,1.26,-0.15,-11.9048,post-market,81.25,103.34,22.09,1
7,2022-06-30,EUR,5.317000e+09,7.516000e+09,2.199000e+09,2.199000e+09,9.650000e+08,530000000.0,1.545000e+09,4.516000e+09,...,2022-07-21,0.96,1.08,-0.12,-11.1111,pre-market,90.72,81.25,-9.47,0
8,2022-03-31,EUR,4.978000e+09,7.077000e+09,2.099000e+09,2.099000e+09,1.053000e+09,504000000.0,1.425000e+09,3.936000e+09,...,2022-04-22,1.00,1.15,-0.15,-13.0435,pre-market,110.96,90.72,-20.24,0
9,2021-12-31,EUR,5.822000e+09,7.980000e+09,2.158000e+09,2.158000e+09,1.470000e+09,757000000.0,1.412000e+09,4.353000e+09,...,2022-01-13,1.86,1.69,0.17,10.0592,pre-market,140.11,110.96,-29.15,0
10,2021-09-30,EUR,4.887000e+09,6.845000e+09,1.958000e+09,1.958000e+09,1.249000e+09,576000000.0,1.300000e+09,3.651000e+09,...,2021-10-12,1.74,1.32,0.42,31.8182,pre-market,135.04,140.11,5.07,1


In [128]:
ccc = calculateFundamentalRatios(merged_df)
ccc

,fiscalDateEnding,reportedCurrency_x,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,Quick Ratio,Debt to Equity,Debt to Asset,Gross Margin,Profit Margin,Operating Margin,Interest Coverage,Inventory Ratio,Return On StockHolder,Free Cash Flow Conversion
1,2023-12-31,EUR,6.208000e+09,8.468000e+09,2.260000e+09,2.260000e+09,1.985000e+09,353000000.0,1.667000e+09,4.223000e+09,...,NaN,0.578115,0.364997,0.733113,0.207959,0.234412,10.784615,NaN,0.030453,0.767703
2,2023-09-30,EUR,5.636000e+09,7.744000e+09,2.108000e+09,2.108000e+09,1.724000e+09,327000000.0,1.515000e+09,3.949000e+09,...,NaN,0.576620,0.364220,0.727789,0.227402,0.258781,7.333333,NaN,0.029749,0.403945
3,2023-06-30,EUR,5.410000e+09,7.555000e+09,2.145000e+09,2.145000e+09,1.358000e+09,322000000.0,1.565000e+09,4.055000e+09,...,NaN,0.655366,0.394297,0.716082,0.144672,0.192323,11.534435,NaN,0.082368,0.394562
4,2023-03-31,EUR,5.285000e+09,7.441000e+09,2.156000e+09,2.156000e+09,1.230000e+09,348000000.0,1.573000e+09,4.220000e+09,...,0.895591,0.758683,0.415283,0.710254,0.090982,0.165300,3.324232,0.291943,0.010112,1.326646
5,2022-12-31,EUR,6.064000e+09,8.436000e+09,2.372000e+09,2.372000e+09,1.328000e+09,517000000.0,1.621000e+09,4.330000e+09,...,0.517534,0.730235,0.406153,0.718824,0.089972,0.157421,1.749260,0.321192,0.008272,1.062813
6,2022-09-30,EUR,5.575000e+09,7.841000e+09,2.266000e+09,2.266000e+09,1.239000e+09,521000000.0,1.574000e+09,4.344000e+09,...,NaN,0.749430,0.411432,0.711006,0.108405,0.158016,2.957746,NaN,0.015576,0.309121
7,2022-06-30,EUR,5.317000e+09,7.516000e+09,2.199000e+09,2.199000e+09,9.650000e+08,530000000.0,1.545000e+09,4.516000e+09,...,NaN,0.822909,0.433847,0.707424,0.071581,0.128393,2.811448,NaN,0.005095,0.022949
8,2022-03-31,EUR,4.978000e+09,7.077000e+09,2.099000e+09,2.099000e+09,1.053000e+09,504000000.0,1.425000e+09,3.936000e+09,...,NaN,0.847077,0.441983,0.703405,0.133531,0.148792,3.645729,NaN,0.018570,1.261015
9,2021-12-31,EUR,5.822000e+09,7.980000e+09,2.158000e+09,2.158000e+09,1.470000e+09,757000000.0,1.412000e+09,4.353000e+09,...,1.154561,0.763030,0.416558,0.729574,0.245238,0.125689,6.883582,3.670068,0.037423,0.690476
10,2021-09-30,EUR,4.887000e+09,6.845000e+09,1.958000e+09,1.958000e+09,1.249000e+09,576000000.0,1.300000e+09,3.651000e+09,...,NaN,0.801239,0.433738,0.713952,0.255515,0.287655,8.454936,NaN,0.039940,0.407731


In [129]:
data = dropUselessValues(ccc,'SAP')
data

,reportedEPS,Action,Current Ratio,Quick Ratio,Debt to Equity,Debt to Asset,Gross Margin,Profit Margin,Operating Margin,Interest Coverage,Inventory Ratio,Return On StockHolder,Free Cash Flow Conversion
1,1.41,1,1.408548,NaN,0.578115,0.364997,0.733113,0.207959,0.234412,10.784615,NaN,0.030453,0.767703
2,1.45,1,1.554552,NaN,0.576620,0.364220,0.727789,0.227402,0.258781,7.333333,NaN,0.029749,0.403945
3,3.14,0,1.421241,NaN,0.655366,0.394297,0.716082,0.144672,0.192323,11.534435,NaN,0.082368,0.394562
4,1.27,1,1.406282,0.895591,0.758683,0.415283,0.710254,0.090982,0.165300,3.324232,0.291943,0.010112,1.326646
5,0.99,1,1.064061,0.517534,0.730235,0.406153,0.718824,0.089972,0.157421,1.749260,0.321192,0.008272,1.062813
6,1.11,1,1.029951,NaN,0.749430,0.411432,0.711006,0.108405,0.158016,2.957746,NaN,0.015576,0.309121
7,0.96,0,0.963004,NaN,0.822909,0.433847,0.707424,0.071581,0.128393,2.811448,NaN,0.005095,0.022949
8,1.00,0,1.111285,NaN,0.847077,0.441983,0.703405,0.133531,0.148792,3.645729,NaN,0.018570,1.261015
9,1.86,0,1.242191,1.154561,0.763030,0.416558,0.729574,0.245238,0.125689,6.883582,3.670068,0.037423,0.690476
10,1.74,1,1.166494,NaN,0.801239,0.433738,0.713952,0.255515,0.287655,8.454936,NaN,0.039940,0.407731


In [76]:
x = 'MSFT'
merged_df = pd.read_csv(f'{x}fundamental_data.csv')
merged_df['fiscalDateEnding'] = pd.to_datetime(merged_df['fiscalDateEnding'])
# type(merged_df.fiscalDateEnding)
for i,rows in merged_df.iterrows():
    if(i-1<0):
        # list_next.append(0)
        time = rows['fiscalDateEnding'].strftime('%Y-%m-%d')